# data cleaning

## Import Packages

In [ ]:
import re
import statistics
import numpy as np
import pandas as pd
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
import findspark  
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession,SQLContext
import pyspark.sql.functions as f
from tqdm import tqdm
tqdm.pandas(desc="Progress")

## Initialize PySpark

In [ ]:
MAX_MEMORY = "32g"
findspark.init()  # Open the pyspark environment
conf = SparkConf()
conf.set('spark.executor.memory', MAX_MEMORY)\
    .set("spark.driver.memory", MAX_MEMORY)
sc = SparkContext(master='local[*]',appName='learning',conf=conf) # Use local masters available
spark = SparkSession.builder.master('local[*]')\
                    .config("spark.sql.execution.arrow.enabled", "true")\
                    .config("spark.executor.memory", MAX_MEMORY) \
                    .config("spark.driver.memory", MAX_MEMORY) \
                    .config("spark.driver.maxResultSize", '0') \
                    .getOrCreate()
import pyspark.pandas as ps

## Load Data

In [ ]:
df=spark.read.option('header','true').csv('../ACSE Data/transactions.csv') \
        .to_pandas_on_spark()

In [ ]:
df.columns

Index(['trans_id', 'trans_dt', 'store_id', 'cust_id', 'prod_id', 'sales_amt',
       'sales_qty', 'sales_wgt'],
      dtype='object')

## Exclude Invalid Customers & Transactions

In [ ]:
ps.set_option('compute.ops_on_diff_frames', True)
df['cus_trans_count']=df.groupby(['cust_id'])['trans_id'].transform(pd.Series.count)
df=df[df['cus_trans_count']<0.8*df['cus_trans_count'].max()]
ps.reset_option('compute.ops_on_diff_frames')

In [ ]:
ps.set_option('compute.ops_on_diff_frames', True)
df['trans_cus_count']=df.groupby(['trans_id'])['cust_id'].transform(pd.Series.count)
df=df[df['trans_cus_count']<0.8*df['trans_cus_count'].max()]
ps.reset_option('compute.ops_on_diff_frames')

## Filter High Demanded Products

In [ ]:
df['sales_amt']=df['sales_amt'].astype(float)

In [ ]:
prod_revenue=df.groupby(['prod_id'])['sales_amt'].sum().sort_values(ascending=False).cumsum()/df['sales_amt'].sum()
prod_revenue.name='cum_amt_pct'

In [ ]:
prod_revenue.to_pandas().to_csv('prod_revenue_all.csv',index=True)

In [ ]:
prod_revenue=pd.read_csv('prod_revenue_all.csv')
prod_revenue=prod_revenue[prod_revenue['cum_amt_pct']<=0.3]
prod_revenue=ps.from_pandas(prod_revenue)

In [ ]:
df[['trans_dt','store_id','prod_id','sales_amt','sales_qty','sales_wgt']] \
    .merge(prod_revenue['prod_id'],on=['prod_id']) \
    .to_csv('high_demanded_prod.csv', num_files=1)

C:\Users\lenovo\anaconda3\lib\site-packages\pyspark\pandas\generic.py:864: FutureWarning: `num_files` has been deprecated and might be removed in a future version. Use `DataFrame.spark.repartition` instead.
  warnings.warn(


# cross elasticity

## Import Packages

In [ ]:
import numpy as np
import pandas as pd
import statistics
from joblib import Parallel, delayed
from tqdm import tqdm
tqdm.pandas(desc="Progress")
pd.set_option('display.max_columns', None)

## Necessary Function

In [ ]:
def pd_to_datetime_parallel(series,format=None):
    table_lis=Parallel(n_jobs=-1,backend='loky') \
                        (delayed(pd.to_datetime)(date,format=None,errors='coerce')
                        for date in tqdm(series))
    return pd.Series(table_lis).values

## Load Data

In [ ]:
df=pd.read_csv('high_demanded_prod.csv',usecols=['trans_dt','prod_id','sales_amt','sales_qty'],
                parse_dates=[0])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378536905 entries, 0 to 378536904
Data columns (total 4 columns):
 #   Column     Dtype         
---  ------     -----         
 0   trans_dt   datetime64[ns]
 1   prod_id    int64         
 2   sales_amt  float64       
 3   sales_qty  int64         
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 11.3 GB


## Clean Data

In [ ]:
df=df[df['sales_qty']>0]

In [ ]:
df=df.groupby(['prod_id','trans_dt'])\
        .agg(daily_amt=('sales_amt','sum'), daily_qty=('sales_qty','sum'))\
        .reset_index()
df.head()

,prod_id,trans_dt,daily_amt,daily_qty
0,20001892,2017-06-24,1285.0,257
1,20001892,2017-06-25,950.0,190
2,20001892,2017-06-26,1165.0,233
3,20001892,2017-06-27,1275.0,255
4,20001892,2017-06-28,1695.0,339


In [ ]:
df.shape

(581412, 4)

In [ ]:
df['price']=df['daily_amt']/df['daily_qty']

In [ ]:
df['YearWeek']=df['trans_dt'].progress_apply(lambda x: '{year}-{week}'.format(year=x.year, week=x.week))

Progress: 100%|██████████| 581412/581412 [00:01<00:00, 339027.02it/s]


In [ ]:
df.head()

,prod_id,trans_dt,daily_amt,daily_qty,price,YearWeek
0,20001892,2017-06-24,1285.0,257,5.0,2017-25
1,20001892,2017-06-25,950.0,190,5.0,2017-25
2,20001892,2017-06-26,1165.0,233,5.0,2017-26
3,20001892,2017-06-27,1275.0,255,5.0,2017-26
4,20001892,2017-06-28,1695.0,339,5.0,2017-26


In [ ]:
df.to_csv('high_demaned_prod_with_price.csv',index=False)

## Weekly Level Data

In [ ]:
df=pd.read_csv('high_demaned_prod_with_price.csv',parse_dates=[0])

In [ ]:
df.head()

,prod_id,trans_dt,daily_amt,daily_qty,price,YearWeek
0,20001892,2017-06-24,1285.0,257,5.0,2017-25
1,20001892,2017-06-25,950.0,190,5.0,2017-25
2,20001892,2017-06-26,1165.0,233,5.0,2017-26
3,20001892,2017-06-27,1275.0,255,5.0,2017-26
4,20001892,2017-06-28,1695.0,339,5.0,2017-26


In [ ]:
df=df.groupby(['prod_id','YearWeek'])\
        .agg(list_price=('price','max'),
             weekly_amt=('daily_amt','sum'), weekly_demand=('daily_qty','sum')) \
        .reset_index()
df['discounted_price']=round(df['weekly_amt']/df['weekly_demand'],2)
df['list_price']=round(df['list_price'],2)
df

,prod_id,YearWeek,list_price,weekly_amt,weekly_demand,discounted_price
0,20001892,2017-25,5.00,2235.00,447,5.00
1,20001892,2017-26,5.00,8785.00,1757,5.00
2,20001892,2017-27,5.00,9285.00,1857,5.00
3,20001892,2017-28,5.00,11690.00,2338,5.00
4,20001892,2017-29,5.00,10430.00,2086,5.00
...,...,...,...,...,...,...
86779,21204739,2020-52,5.99,36419.19,6081,5.99
86780,21204739,2020-6,5.99,51333.70,8570,5.99
86781,21204739,2020-7,9.83,33442.84,4900,6.83
86782,21204739,2020-8,11.14,51479.47,8167,6.30


In [ ]:
df['year']=df['YearWeek'].progress_apply(lambda x:x[:4])
df['week']=df['YearWeek'].progress_apply(lambda x:x[5:])

Progress: 100%|██████████| 86784/86784 [00:00<00:00, 1046980.51it/s]


In [ ]:
df.sort_values(by=['prod_id','year','week'],inplace=True)

In [ ]:
df['discounted_price_pct']=df.groupby(['prod_id'])['discounted_price'].pct_change()
df['weekly_demand_pct']=df.groupby(['prod_id'])['weekly_demand'].pct_change()

In [ ]:
df['is_promotion']=df.progress_apply(lambda x: 1 if x['discounted_price']<x['list_price'] else 0,axis=1)

Progress: 100%|██████████| 86784/86784 [00:01<00:00, 76926.33it/s]


In [ ]:
df['is_EDLP']=df.groupby(['prod_id'])['is_promotion'].transform(sum)

In [ ]:
df['discount_level']=(df['list_price']-df['discounted_price'])/df['list_price']

In [ ]:
df.head()

,prod_id,YearWeek,list_price,weekly_amt,weekly_demand,discounted_price,year,week,discounted_price_pct,weekly_demand_pct,is_promotion,is_EDLP,discount_level
0,20001892,2017-25,5.0,2235.0,447,5.0,2017,25,NaN,NaN,0,0,0.0
1,20001892,2017-26,5.0,8785.0,1757,5.0,2017,26,0.0,2.930649,0,0,0.0
2,20001892,2017-27,5.0,9285.0,1857,5.0,2017,27,0.0,0.056915,0,0,0.0
3,20001892,2017-28,5.0,11690.0,2338,5.0,2017,28,0.0,0.259020,0,0,0.0
4,20001892,2017-29,5.0,10430.0,2086,5.0,2017,29,0.0,-0.107784,0,0,0.0


In [ ]:
df.to_csv('weekly_level_data.csv',index=False)

## Cross-elasticity

In [ ]:
df=pd.read_csv('weekly_level_data.csv')

In [ ]:
df_pairs=pd.merge(df[['prod_id','YearWeek','weekly_demand_pct']],
                    df[['prod_id','YearWeek','discounted_price_pct','discounted_price']],
                    on=['YearWeek'])
df_pairs=df_pairs.query('prod_id_x<prod_id_y')

In [ ]:
df_pairs['cross_elasticity']=df_pairs['weekly_demand_pct']/df_pairs['discounted_price_pct']
df_pairs['cross_elasticity']=df_pairs['cross_elasticity'].replace([np.inf, -np.inf], np.nan)

In [ ]:
df_pairs

,prod_id_x,YearWeek,weekly_demand_pct,prod_id_y,discounted_price_pct,discounted_price,cross_elasticity
1,20001892,2017-25,NaN,20002646001,NaN,6.62,NaN
2,20001892,2017-25,NaN,20003839001,NaN,2.18,NaN
3,20001892,2017-25,NaN,20005739,NaN,5.88,NaN
4,20001892,2017-25,NaN,20005858,NaN,4.99,NaN
5,20001892,2017-25,NaN,20007535001,NaN,2.82,NaN
...,...,...,...,...,...,...,...
41408168,21204739,2020-20,0.915792,21090447001,-0.014587,6.08,-62.782623
41408169,21204739,2020-20,0.915792,21092325001,0.028609,7.91,32.011090
41408172,21204739,2020-20,0.915792,21097012001,0.151163,1.98,6.058316
41408173,21204739,2020-20,0.915792,21108053001,-0.203593,3.99,-4.498154


In [ ]:
df_pairs['is_substitute']=df_pairs['cross_elasticity'].progress_apply(lambda x: 1 if x>0 else 0)
df_pairs['is_complement']=df_pairs['cross_elasticity'].progress_apply(lambda x: 1 if x<0 else 0)

Progress: 100%|██████████| 20660699/20660699 [00:22<00:00, 910227.90it/s] 


In [ ]:
df_pairs['max_cross_elasticity']=df_pairs.groupby(['prod_id_x','YearWeek'])['cross_elasticity'] \
                                            .transform(max)
df_pairs['competitor_price']=df_pairs.progress_apply(lambda x: x['discounted_price'] 
                                                                if x['max_cross_elasticity']==x['cross_elasticity'] 
                                                                else 0, axis=1)
df_pairs['competitor_price']=df_pairs.groupby(['prod_id_x','YearWeek'])['competitor_price'] \
                                            .transform(max)                                                               

Progress: 100%|██████████| 20660699/20660699 [04:41<00:00, 73455.36it/s]


In [ ]:
df_pairs

,prod_id_x,YearWeek,weekly_demand_pct,prod_id_y,discounted_price_pct,discounted_price,cross_elasticity,is_substitute,is_complement,max_cross_elasticity,competitor_price
1,20001892,2017-25,NaN,20002646001,NaN,6.62,NaN,0,0,NaN,0.00
2,20001892,2017-25,NaN,20003839001,NaN,2.18,NaN,0,0,NaN,0.00
3,20001892,2017-25,NaN,20005739,NaN,5.88,NaN,0,0,NaN,0.00
4,20001892,2017-25,NaN,20005858,NaN,4.99,NaN,0,0,NaN,0.00
5,20001892,2017-25,NaN,20007535001,NaN,2.82,NaN,0,0,NaN,0.00
...,...,...,...,...,...,...,...,...,...,...,...
41408168,21204739,2020-20,0.915792,21090447001,-0.014587,6.08,-62.782623,0,1,300.83764,6.59
41408169,21204739,2020-20,0.915792,21092325001,0.028609,7.91,32.011090,1,0,300.83764,6.59
41408172,21204739,2020-20,0.915792,21097012001,0.151163,1.98,6.058316,1,0,300.83764,6.59
41408173,21204739,2020-20,0.915792,21108053001,-0.203593,3.99,-4.498154,0,1,300.83764,6.59


In [ ]:
df_substitute=df_pairs.query('is_substitute>0').groupby(['prod_id_x','YearWeek'])\
                        .agg(substitute_price=('discounted_price','mean')).reset_index()
df_substitute

,prod_id_x,YearWeek,substitute_price
0,20001892,2017-26,7.261495
1,20001892,2017-27,44.256554
2,20001892,2017-28,7.942552
3,20001892,2017-29,17.812929
4,20001892,2017-30,101.039412
...,...,...,...
85512,21108053001,2020-48,6.930000
85513,21108053001,2020-49,6.920000
85514,21108053001,2020-52,6.930000
85515,21108053001,2020-6,4.990000


In [ ]:
df_complement=df_pairs.query('is_complement>0').groupby(['prod_id_x','YearWeek'])\
                        .agg(complement_price=('discounted_price','mean')).reset_index()
df_complement

,prod_id_x,YearWeek,complement_price
0,20001892,2017-26,8.565272
1,20001892,2017-27,7.445517
2,20001892,2017-28,38.590053
3,20001892,2017-29,8.369198
4,20001892,2017-30,7.508678
...,...,...,...
85502,21108053001,2020-47,6.910000
85503,21108053001,2020-5,4.980000
85504,21108053001,2020-50,6.930000
85505,21108053001,2020-51,6.910000


In [ ]:
df_competitor=df_pairs.query('competitor_price>0').groupby(['prod_id_x','YearWeek'])\
                        .agg(competitor_price=('competitor_price','max')).reset_index()
df_competitor

,prod_id_x,YearWeek,competitor_price
0,20001892,2017-26,15.28
1,20001892,2017-27,15.77
2,20001892,2017-28,9.81
3,20001892,2017-29,11.88
4,20001892,2017-30,12.98
...,...,...,...
86054,21108053001,2020-51,6.91
86055,21108053001,2020-52,6.93
86056,21108053001,2020-6,4.99
86057,21108053001,2020-7,4.98


In [ ]:
df=df.merge(df_substitute,left_on=['prod_id','YearWeek'],right_on=['prod_id_x','YearWeek'],how='left')
df=df.merge(df_complement,left_on=['prod_id','YearWeek'],right_on=['prod_id_x','YearWeek'],how='left')
df=df.merge(df_competitor,left_on=['prod_id','YearWeek'],right_on=['prod_id_x','YearWeek'],how='left')
df.drop(columns=['prod_id_x_x','prod_id_x_y','prod_id_x'],inplace=True)

In [ ]:
df

,prod_id,YearWeek,list_price,weekly_amt,weekly_demand,discounted_price,year,week,discounted_price_pct,weekly_demand_pct,is_promotion,is_EDLP,discount_level,substitute_price,complement_price,competitor_price
0,20001892,2017-25,5.00,2235.00,447,5.00,2017,25,NaN,NaN,0,0,0.000000,NaN,NaN,NaN
1,20001892,2017-26,5.00,8785.00,1757,5.00,2017,26,0.000000,2.930649,0,0,0.000000,7.261495,8.565272,15.28
2,20001892,2017-27,5.00,9285.00,1857,5.00,2017,27,0.000000,0.056915,0,0,0.000000,44.256554,7.445517,15.77
3,20001892,2017-28,5.00,11690.00,2338,5.00,2017,28,0.000000,0.259020,0,0,0.000000,7.942552,38.590053,9.81
4,20001892,2017-29,5.00,10430.00,2086,5.00,2017,29,0.000000,-0.107784,0,0,0.000000,17.812929,8.369198,11.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86779,21204739,2020-52,5.99,36419.19,6081,5.99,2020,52,0.001672,0.087835,0,66,0.000000,5.080891,5.206481,6.90
86780,21204739,2020-6,5.99,51333.70,8570,5.99,2020,6,0.000000,0.409308,0,66,0.000000,5.439863,4.743394,4.50
86781,21204739,2020-7,9.83,33442.84,4900,6.83,2020,7,0.140234,-0.428238,1,66,0.305188,4.617436,5.056374,6.97
86782,21204739,2020-8,11.14,51479.47,8167,6.30,2020,8,-0.077599,0.666735,1,66,0.434470,5.339718,4.628750,6.98


In [ ]:
df.to_csv('weekly_level_data_with_cross_elasticity.csv',index=False)

# Target Products and Optimal Price

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv('weekly_level_data_with_cross_elasticity.csv')

In [ ]:
data.shape

(86784, 16)

In [ ]:
data = data.dropna()

In [ ]:
data.shape

(85267, 16)

In [ ]:
data['weekly_demand_max'] = data.groupby('prod_id')['weekly_demand'].transform(np.max)
data['weekly_demand_max'] = data['weekly_demand_max'] * 1.1
data['demand_transformed'] = np.log(data['weekly_demand'] / (data['weekly_demand_max'] - data['weekly_demand']))

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso

In [ ]:
prod = []
coef0 = []
coef1 = []
coef2 = []
coef3 = []
coef4 = []
intercept = []
for i in data['prod_id'].unique():
    prod_data = data[data['prod_id'] == i]
    X = prod_data[['discounted_price', 'substitute_price', 'complement_price', 'week', 'discount_level']]
    y = prod_data['demand_transformed']
    model = Ridge(alpha=1, fit_intercept=True)
    model.fit(X, y)
    coef0.append(model.coef_[0])
    coef1.append(model.coef_[1])
    coef2.append(model.coef_[2])
    coef3.append(model.coef_[3])
    coef4.append(model.coef_[4])
    intercept.append(model.intercept_)
    prod.append(i)

In [ ]:
coefs = pd.DataFrame({'prod':prod, 'coef0':coef0, 'coef1':coef1, 'coef2':coef2, 'coef3':coef3, 'coef4':coef4, 'intercept':intercept})

In [ ]:
coefs

,prod,coef0,coef1,coef2,coef3,coef4,intercept
0,20001892,0.000000,0.004537,0.006521,0.000013,0.000000,-0.953165
1,20002646001,0.652361,-0.031779,-0.460871,0.005587,-9.087702,-2.440749
2,20003839001,-0.387083,0.181794,-0.168142,0.000602,-0.503278,1.300225
3,20005739,0.043266,-0.002035,0.001173,-0.006661,0.063773,0.336101
4,20005858,-1.091803,0.007975,0.007134,-0.010503,0.128716,5.240355
...,...,...,...,...,...,...,...
506,21179127,0.456252,0.032552,-0.722807,-0.003424,1.052833,0.052777
507,21184958,0.007147,0.711497,0.426715,-0.005295,-0.002536,-6.388337
508,21189812,-0.335201,0.682139,-0.338870,-0.002441,-0.344683,-0.134511
509,21204386,-0.188538,0.425274,0.348922,-0.021952,0.087895,-1.835372


In [ ]:
data_avg = data.groupby('prod_id').mean().reset_index()

In [ ]:
data_avg

,prod_id,list_price,weekly_amt,weekly_demand,discounted_price,year,week,discounted_price_pct,weekly_demand_pct,is_promotion,is_EDLP,discount_level,substitute_price,complement_price,competitor_price,weekly_demand_max,demand_transformed
0,20001892,5.000000,12530.606936,2506.121387,5.000000,2018.676301,28.687861,0.000000,0.882282,0.000000,0.0,0.000000,16.554868,20.065922,16.397457,7418.4,-0.746813
1,20005739,5.898174,12389.607391,2153.582609,5.753739,2018.113043,26.947826,0.002106,0.240053,0.973913,113.0,0.024929,21.593767,25.641002,15.895130,3614.6,0.393240
2,20005858,4.719227,8989.521326,2015.889503,4.511436,2018.723757,28.254144,0.002331,0.057274,0.817680,148.0,0.043141,18.854326,17.112304,14.961713,3564.0,0.296000
3,20009282,7.403966,8769.790168,1324.497207,6.847654,2018.715084,28.033520,0.009738,0.079617,0.726257,132.0,0.073588,19.002512,17.239062,15.806201,3834.6,-0.676316
4,20011985,5.407972,12029.387483,2394.965035,5.166154,2018.384615,27.573427,0.004594,5.550322,0.993007,143.0,0.042558,19.935677,21.381089,14.960280,3749.9,0.425396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506,21035960001,6.218090,27825.675393,4974.404494,5.853483,2019.123596,26.831461,0.013571,7.353239,0.977528,129.0,0.056870,4.541339,4.210478,4.757079,11056.1,-0.431315
507,21089853001,4.345638,11141.682234,2560.968085,4.309149,2019.223404,27.404255,0.003659,1.840418,0.797872,106.0,0.008030,4.473316,4.548422,4.629362,4336.2,0.360929
508,21090447001,6.273500,24146.425833,4056.600000,5.970833,2019.483333,27.600000,0.008101,69.736310,0.933333,116.0,0.045765,4.119278,4.287611,4.268333,8191.7,-0.398514
509,21092325001,7.821579,14035.090789,1916.947368,7.583158,2019.578947,27.078947,0.011145,1.120177,0.868421,70.0,0.030360,3.369605,3.622500,3.452632,6633.0,-1.154086


In [ ]:
coefs[coefs['prod'] == i]

,prod,coef0,coef1,coef2,coef3,coef4,intercept
510,21204739,-0.095426,0.541575,0.07252,-0.021867,-0.074609,-3.102101


In [ ]:
data_avg[data_avg['prod_id'] == 21204739]

,prod_id,list_price,weekly_amt,weekly_demand,discounted_price,year,week,discounted_price_pct,weekly_demand_pct,is_promotion,is_EDLP,discount_level,substitute_price,complement_price,competitor_price,weekly_demand_max,demand_transformed
321,21204739,9.172394,60817.356761,8630.295775,7.092817,2019.732394,30.915493,0.020755,0.389069,0.915493,66.0,0.199914,5.058781,4.964984,7.629718,38150.2,-1.370102


In [ ]:
data_avg[data_avg['prod_id'] == 21204739].iloc[0, 4]

7.092816901408451

In [ ]:
prod = []
price = []
elasticity = []

for i in data['prod_id'].unique():
    a = coefs[coefs['prod'] == i].iloc[0, 6]
    b = coefs[coefs['prod'] == i].iloc[0, 1]
    c = coefs[coefs['prod'] == i].iloc[0, 2]
    d = coefs[coefs['prod'] == i].iloc[0, 3]
    e = coefs[coefs['prod'] == i].iloc[0, 4]
    f = coefs[coefs['prod'] == i].iloc[0, 5]

    substitute_price = data_avg[data_avg['prod_id'] == i].iloc[0, 12]
    complement_price = data_avg[data_avg['prod_id'] == i].iloc[0, 13]
    week = data_avg[data_avg['prod_id'] == i].iloc[0, 6]
    discount_level = data_avg[data_avg['prod_id'] == i].iloc[0, 4]

    for p in np.linspace(0, 100, 100):
        p = round(p, 2)
        coef_sum = a + b * p + c * substitute_price + d * complement_price + e * week + f * discount_level
        elasticity.append(((b * p)) / (1 + np.exp(-(coef_sum))))
        price.append(p)
        prod.append(i)

In [ ]:
price_elasticity = pd.DataFrame({'prod_id':prod, 'price':price, 'elasticity':elasticity})
price_elasticity['dist'] = np.abs(price_elasticity['elasticity'] - (-1))

In [ ]:
optimal_price = pd.DataFrame()
for i in price_elasticity['prod_id'].unique():
    optimal = price_elasticity[price_elasticity['prod_id'] == i].sort_values('dist').head(1)
    optimal_price = pd.concat([optimal_price, optimal], axis=0)

In [ ]:
result=optimal_price[(optimal_price['price'] != 0) & (optimal_price['price'] != 100)]
result=result[result['dist']<0.15]
result

,prod_id,price,elasticity,dist
407,20005858,7.07,-0.984984,0.015016
503,20007535001,3.03,-0.917678,0.082322
1102,20020516001,2.02,-0.982679,0.017321
1415,20026161001,15.15,-0.850682,0.149318
1563,20026703001,63.64,-0.987384,0.012616
...,...,...,...,...
49001,21058519,1.01,-1.100896,0.100896
49426,21087812,26.26,-0.990247,0.009753
49709,21090447001,9.09,-0.990155,0.009845
50103,21097012001,3.03,-0.984295,0.015705


In [ ]:
prod = []
demand_transformed = []
price = []

for i in result['prod_id'].unique():
    a = coefs[coefs['prod'] == i].iloc[0, 6]
    b = coefs[coefs['prod'] == i].iloc[0, 1]
    c = coefs[coefs['prod'] == i].iloc[0, 2]
    d = coefs[coefs['prod'] == i].iloc[0, 3]
    e = coefs[coefs['prod'] == i].iloc[0, 4]
    f = coefs[coefs['prod'] == i].iloc[0, 5]

    substitute_price = data_avg[data_avg['prod_id'] == i].iloc[0, 12]
    complement_price = data_avg[data_avg['prod_id'] == i].iloc[0, 13]
    week = data_avg[data_avg['prod_id'] == i].iloc[0, 6]
    discount_level = data_avg[data_avg['prod_id'] == i].iloc[0, 4]

    p = result[result['prod_id'] == i].iloc[0, 1]
    price.append(p)

    demand_transformed.append(b * p + c * substitute_price + d * complement_price + e * week + f * discount_level + a)
    prod.append(i)

In [ ]:
demand_pred = pd.merge(pd.DataFrame({'prod_id':prod, 'demand_transformed':demand_transformed, 'price':price}), data[['prod_id', 'weekly_demand_max']].drop_duplicates())
demand_pred['demand'] = demand_pred['weekly_demand_max'] / ((1 / np.exp(demand_pred['demand_transformed'])) + 1)
demand_pred['revenue'] = round(demand_pred['demand'] * demand_pred['price'],2)
demand_pred

,prod_id,demand_transformed,price,weekly_demand_max,demand,revenue
0,20005858,-1.922308,7.07,3564.0,454.782106,3215.31
1,20007535001,2.283285,3.03,41838.5,37967.737212,115042.24
2,20020516001,0.618591,2.02,12588.4,8181.176991,16525.98
3,20026161001,-0.153158,15.15,12944.8,5977.715979,90562.40
4,20026703001,-2.060442,63.64,39768.3,4493.876745,285990.32
...,...,...,...,...,...,...
229,21058519,6.346727,1.01,3611.3,3604.982364,3641.03
230,21087812,3.326307,26.26,5275.6,5092.644015,133732.83
231,21090447001,-2.031616,9.09,8191.7,949.608456,8631.94
232,21097012001,0.237860,3.03,108846.1,60865.236970,184421.67


In [ ]:
product=pd.merge(data[['prod_id','list_price','discounted_price']].groupby(['prod_id']).mean()\
                                                                    .reset_index(),result,on=['prod_id'])

In [ ]:
product['increase_price']=product.apply(lambda x: 1 if x['price']>x['discounted_price'] else 0, axis=1)

In [ ]:
product['price_pct']=(product['price']-product['discounted_price'])/product['discounted_price']

In [ ]:
product['increase_price'].sum()

139

In [ ]:
product=product.merge(demand_pred[['prod_id','demand','revenue']],on='prod_id',how='left')
product

,prod_id,list_price,discounted_price,price,elasticity,dist,increase_price,price_pct,revenue
0,20005858,4.719227,4.511436,7.07,-0.984984,0.015016,1,0.567128,3215.31
1,20035921,3.880986,3.806690,7.07,-0.915326,0.084674,1,0.857256,15512.96
2,20038148,22.273039,20.071381,4.04,-1.090981,0.090981,0,-0.798718,10741.64
3,20049221,9.333094,8.672044,24.24,-1.040411,0.040411,1,1.795189,56143.11
4,20055009,10.698398,8.954033,4.04,-0.866869,0.133131,0,-0.548807,9813.68
...,...,...,...,...,...,...,...,...,...
229,21027142001,5.427132,5.269380,33.33,-1.059612,0.059612,1,5.325223,128461.20
230,21030970001,8.377803,8.091515,21.21,-1.021684,0.021684,1,1.621264,20612.50
231,21035960001,6.218090,5.853483,1.01,-1.042866,0.042866,0,-0.827453,9553.41
232,21090447001,6.273500,5.970833,9.09,-0.990155,0.009845,1,0.522401,8631.94


In [ ]:
product=product.sort_values(by=['increase_price','revenue'],ascending=True)
product=product.iloc[45:,]
product

,prod_id,list_price,discounted_price,price,elasticity,dist,increase_price,price_pct,revenue
87,20801192,16.564917,14.487680,14.14,-0.965160,0.034840,0,-0.023998,14390.52
22,20126276,4.449558,4.336685,2.02,-0.927214,0.072786,0,-0.534206,14973.60
181,20310940001,4.620442,4.243757,3.03,-1.070053,0.070053,0,-0.286010,15869.00
137,20020516001,2.204833,2.151167,2.02,-0.982679,0.017321,0,-0.060975,16525.98
99,20902733,10.118177,9.050552,2.02,-0.913226,0.086774,0,-0.776809,16755.67
...,...,...,...,...,...,...,...,...,...
168,20152465001,4.118013,3.744872,60.61,-1.010162,0.010162,1,15.184800,201221.77
166,20145621001,3.367624,3.091989,5.05,-1.030633,0.030633,1,0.633253,274934.39
139,20026703001,3.848729,3.475635,63.64,-0.987384,0.012616,1,17.310321,285990.32
151,20080137001,5.290110,4.743591,31.31,-1.043071,0.043071,1,5.600485,422185.04


In [ ]:
product=product.sort_values(by=['increase_price','revenue'],ascending=[False,True])
product=product.iloc[-150:,]
product

,prod_id,list_price,discounted_price,price,elasticity,dist,increase_price,price_pct,revenue
67,20602264,6.054309,5.599834,22.22,-0.968544,0.031456,1,2.967975,12596.74
225,20905336001,6.243731,5.907761,14.14,-0.856130,0.143870,1,1.393462,12675.56
69,20612257,2.981271,2.804751,3.03,-1.122114,0.122114,1,0.080310,12900.85
207,20690699001,5.697238,5.385801,12.12,-0.943646,0.056354,1,1.250362,14215.64
12,20078939,3.866028,3.793121,5.05,-0.880521,0.119479,1,0.331358,14236.02
...,...,...,...,...,...,...,...,...,...
20,20121871,17.427722,15.342111,5.05,-0.913799,0.086201,0,-0.670841,117382.70
107,20964482,22.434681,19.557447,10.10,-0.980248,0.019752,0,-0.483573,180490.47
115,20972941,32.411068,25.627573,8.08,-1.022163,0.022163,0,-0.684715,182808.38
90,20812304,39.599841,28.077143,7.07,-0.975434,0.024566,0,-0.748194,203368.55


In [ ]:
product.to_csv('final_product.csv',index=False)

# Expected changes

## Import Packages

In [ ]:
import numpy as np
import pandas as pd
import statistics
from joblib import Parallel, delayed
from tqdm import tqdm
tqdm.pandas(desc="Progress")
pd.set_option('display.max_columns', None)

## Load Data

In [ ]:
df=pd.read_csv('high_demanded_prod.csv',usecols=['trans_dt','prod_id','store_id','sales_amt','sales_qty'],
                parse_dates=[0])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378536905 entries, 0 to 378536904
Data columns (total 5 columns):
 #   Column     Dtype         
---  ------     -----         
 0   trans_dt   datetime64[ns]
 1   store_id   int64         
 2   prod_id    int64         
 3   sales_amt  float64       
 4   sales_qty  int64         
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 14.1 GB


## Clean Data

In [ ]:
df=df[df['sales_qty']>0]

In [ ]:
df=df.groupby(['prod_id','store_id'])\
        .agg(total_amt=('sales_amt','sum'),total_qty=('sales_qty','sum'),
                sales_num_date=('trans_dt','nunique')).reset_index()
df['yearly_amt']=round(df['total_amt']/df['sales_num_date']*365,2)
df['yearly_qty']=round(df['total_qty']/df['sales_num_date']*365,2)
df.head()

,prod_id,store_id,total_amt,sales_num_date,yearly_amt
0,20001892,1000,13725.0,723,6928.941909
1,20001892,1001,60480.0,1106,19959.493671
2,20001892,1003,47040.0,1106,15524.050633
3,20001892,1004,35970.0,934,14056.798715
4,20001892,1005,75935.0,1127,24592.968057


In [ ]:
product=pd.read_csv('final_product.csv')
product

,prod_id,list_price,discounted_price,price,elasticity,dist,increase_price,price_pct,revenue
0,20602264,6.054309,5.599834,22.22,-0.968544,0.031456,1,2.967975,12596.74
1,20905336001,6.243731,5.907761,14.14,-0.856130,0.143870,1,1.393462,12675.56
2,20612257,2.981271,2.804751,3.03,-1.122114,0.122114,1,0.080310,12900.85
3,20690699001,5.697238,5.385801,12.12,-0.943646,0.056354,1,1.250362,14215.64
4,20078939,3.866028,3.793121,5.05,-0.880521,0.119479,1,0.331358,14236.02
...,...,...,...,...,...,...,...,...,...
145,20121871,17.427722,15.342111,5.05,-0.913799,0.086201,0,-0.670841,117382.70
146,20964482,22.434681,19.557447,10.10,-0.980248,0.019752,0,-0.483573,180490.47
147,20972941,32.411068,25.627573,8.08,-1.022163,0.022163,0,-0.684715,182808.38
148,20812304,39.599841,28.077143,7.07,-0.975434,0.024566,0,-0.748194,203368.55


In [ ]:
data=df.merge(product[['prod_id','demand','revenue']],on='prod_id',how='right')

In [ ]:
data['revenue']=round(data['revenue']/7*365,2)
data['demand']=round(data['demand']/7*365,2)

In [ ]:
data['total_prod_yearly_amt']=data.groupby(['prod_id'])['yearly_amt'].transform(np.sum)
data['total_prod_yearly_amt']=round(data['total_prod_yearly_amt'],2)

In [ ]:
data['total_prod_yearly_qty']=data.groupby(['prod_id'])['yearly_qty'].transform(np.sum)
data['total_prod_yearly_qty']=round(data['total_prod_yearly_qty'],2)

In [ ]:
data['actual_amt_pct']=data['yearly_amt']/data['total_prod_yearly_amt']
data['actual_qty_pct']=data['yearly_qty']/data['total_prod_yearly_qty']

In [ ]:
data['predicted_prod_revenue']=round(data['actual_amt_pct']*data['revenue'],2)
data['predicted_prod_demand']=round(data['actual_qty_pct']*data['demand'],2)

In [ ]:
data=data[['prod_id','store_id','yearly_amt','predicted_prod_revenue','yearly_qty','predicted_prod_demand']].copy()
data

,prod_id,store_id,yearly_amt,predicted_prod_revenue
0,20602264,1000,10370.234656,10537.10
1,20602264,1001,4270.686860,4339.40
2,20602264,1003,12004.518750,12197.68
3,20602264,1004,27391.954725,27832.71
4,20602264,1005,15292.491359,15538.56
...,...,...,...,...
6959,20182566001,1194,91787.932765,269044.59
6960,20182566001,1200,43064.238314,126227.93
6961,20182566001,1208,36696.011636,107561.67
6962,20182566001,1212,142361.107554,417282.37


In [ ]:
store_result=data.groupby(['store_id']).agg(original_revenue=('yearly_amt','sum'),
                                            optimal_revenue=('predicted_prod_revenue','sum'),
                                            original_demand=('yearly_qty','sum'),
                                            optimal_demand=('predicted_prod_demand','sum'))

In [ ]:
store_result['original_demand']=round(store_result['original_demand'],2)
store_result['demand_increase']=store_result['optimal_demand']-store_result['original_demand']
store_result['demand_increase_pct']=round(store_result['demand_increase']/store_result['original_demand'],2)
store_result

In [ ]:
store_result['original_revenue']=round(store_result['original_revenue'],2)
store_result['revenue_increase']=store_result['optimal_revenue']-store_result['original_revenue']
store_result['revenue_increase_pct']=round(store_result['revenue_increase']/store_result['original_revenue'],2)
store_result

,original_revenue,optimal_revenue,revenue_increase,revenue_increase_pct
store_id,,,,
1000,6750850.23,10493656.24,3742806.01,0.554420
1001,7489001.44,11129434.38,3640432.94,0.486104
1003,5477654.50,8088208.34,2610553.84,0.476582
1004,10774234.57,16417284.54,5643049.97,0.523754
1005,6557900.35,9517588.49,2959688.14,0.451316
1007,9756051.17,15368114.44,5612063.27,0.575239
1010,6670866.77,10565164.16,3894297.39,0.583777
1011,8765478.45,13431917.30,4666438.85,0.532366
1014,7028444.15,10754010.42,3725566.27,0.530070


In [ ]:
store_result['original_profit']=round(store_result['original_revenue']*0.2,2)
store_result['optimal_profit']=round(store_result['optimal_revenue']*0.2,2)
store_result['profit_increase']=round(store_result['revenue_increase']*0.2,2)
store_result['profit_increase_pct']=round(store_result['profit_increase']/store_result['original_profit'],2)
store_result

In [ ]:
store_result.to_csv('store_result.csv',index=False)